In [1]:
import os
import sys
import numpy as np
import cv2
import sklearn
import fnmatch
from matplotlib import pyplot as plt
import imutils
sys.path.insert(0, os.path.abspath('../..'))

file_under_use = 'final_set_unique/'
imagePixelSize = 512

In [2]:
mushroomNames = []
for mushroomName in os.listdir('../../MushroomImage/' + file_under_use):
    mushroomNames.append(mushroomName)

In [3]:
numImages = 0
for mushroomName in mushroomNames:
    for imageName in os.listdir('../../MushroomImage/' + file_under_use + mushroomName):
        numImages += 1
print(numImages)

445


In [4]:
index = 0
testCount = 0
trainCount = 0
for mushroomName in mushroomNames:
    dirListing = os.listdir('../../MushroomImage/' + file_under_use + mushroomName)
    numImages = len(dirListing) 
    train = int(numImages * (3/4))-1
    count = 0
    for imageName in os.listdir('../../MushroomImage/' + file_under_use + mushroomName):
        
        if count <= train:
            trainCount += 1
        else:
            testCount += 1
        index += 1
        count += 1
        
print(testCount)
print(trainCount)

106
339


In [10]:
images_train = np.empty([numImages, imagePixelSize+1]) 


def extract_color_histogram(image, bins=(8, 8, 8)):
    # extract a 3D color histogram from the HSV color space using
    # the supplied number of `bins` per channel
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
        [0, 180, 0, 256, 0, 256])
    # handle normalizing the histogram if we are using OpenCV 2.4.X
    if imutils.is_cv2():
        hist = cv2.normalize(hist)
    # otherwise, perform "in place" normalization in OpenCV 3 (I
    # personally hate the way this is done
    else:
        cv2.normalize(hist, hist)
    # return the flattened histogram as the feature vector
    return hist.flatten()

def extract_lines(image):
    median_intensity = np.median(image)
    image = cv2.resize(image, (40,40))
    lower_threshold = int(max(0, (1.0 - 0.33) * median_intensity))
    upper_threshold = int(max(255, (1.0 + 0.33) * median_intensity))
    image = cv2.Canny(image, lower_threshold, upper_threshold)
    zoned_image = np.empty([4,4])
    plt.imshow(image, cmap="gray")
    plt.show()
    for i in range(0,4):
        for j in range(0, 4):
            image_small = image[i*10:i*10+10,j*10:j*10+10]
            zoned_image[i,j] = np.count_nonzero(image_small)
    return zoned_image.flatten()
    




In [11]:
for mushroomName in mushroomNames:
    dirListing = os.listdir('../../MushroomImage/' + file_under_use + mushroomName)
    numImages = len(dirListing) 
    train = int(numImages * (3/4))
    count = 0
    for imageName in os.listdir('../../MushroomImage/' + file_under_use + mushroomName):
        
        image = cv2.imread('../../MushroomImage/' + file_under_use + mushroomName + '/' + imageName, cv2.IMREAD_COLOR)
        #print(mushroomName + '/' + imageName)
        
        if count <= train:
            images_train[train_index] = np.append(extract_color_histogram(image), np.array([mushroomNames.index(mushroomName)]), axis = 0)
            train_index += 1
        else:
            images_test[test_index] = np.append(extract_color_histogram(image), np.array([mushroomNames.index(mushroomName)]), axis = 0)
            test_index += 1
        count += 1

IndexError: index 339 is out of bounds for axis 0 with size 50

In [8]:
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

mushroom_train_data = images_train[:,:imagePixelSize]
mushroom_train_target = images_train[:,-1]
mushroom_test_data = images_test[:,:imagePixelSize]
mushroom_test_target = images_test[:,-1]




#pca = PCA(n_components=0.5, whiten=True)
#pca.fit(mushroom_train_data)
#mushroom_train_data = pca.transform(mushroom_train_data)
#mushroom_test_data = pca.transform(mushroom_test_data)


#scaler = StandardScaler()
#scaler.fit(mushroom_train_data)
#mushroom_train_data = scaler.transform(mushroom_train_data)
#mushroom_test_data = scaler.transform(mushroom_test_data)

#knn = tree.DecisionTreeClassifier()

#knn = KNeighborsClassifier(n_neighbors=7)
from sklearn.ensemble import RandomForestClassifier
knn = RandomForestClassifier()


knn.fit(mushroom_train_data, mushroom_train_target)
print(knn.get_params().keys())

knnScore = knn.score(mushroom_test_data,mushroom_test_target)
print(knnScore)
predict = knn.predict(mushroom_test_data)

for i in np.arange(mushroom_test_target.shape[0]):
    print(mushroomNames[int(predict[i])] + "    " + mushroomNames[int(mushroom_test_target[i])])
print(mushroom_test_target)

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])
0.5849056603773585
11069_Boletus_edulis    10056_Agaricus_arvensis
10056_Agaricus_arvensis    10056_Agaricus_arvensis
12587_Cortinarius_rubellus    10056_Agaricus_arvensis
10056_Agaricus_arvensis    10056_Agaricus_arvensis
10056_Agaricus_arvensis    10056_Agaricus_arvensis
10056_Agaricus_arvensis    10056_Agaricus_arvensis
10056_Agaricus_arvensis    10056_Agaricus_arvensis
10056_Agaricus_arvensis    10056_Agaricus_arvensis
11069_Boletus_edulis    10056_Agaricus_arvensis
10056_Agaricus_arvensis    10056_Agaricus_arvensis
10056_Agaricus_arvensis    10056_Agaricus_arvensis
10056_Agaricus_arvensis    10056_Agaricus_arvensis
10056_Agaricus_arvensis    10056_Agaricus_arven

In [92]:
from numpy import savetxt
savetxt('../mushroom_images/mushroom_image_train_color_data.csv', mushroom_train_data, delimiter=',')
savetxt('../mushroom_images/mushroom_image_train_color_target.csv', mushroom_train_target, delimiter=',')
savetxt('../mushroom_images/mushroom_image_test_color_data.csv', mushroom_test_data, delimiter=',')
savetxt('../mushroom_images/mushroom_image_test_color_target.csv', mushroom_test_target, delimiter=',')

## deletes images

print(image.flatten().shape)
for mushroomName in mushroomNames:
    for imageName in os.listdir('../../MushroomImage/' + file_under_use + mushroomName):
        for imageName in os.listdir('../../MushroomImage/' + file_under_use + mushroomName):
            os.remove('../../MushroomImage/' + file_under_use + mushroomName + '/' + imageName)
        


## Removes folders with less than 15 images

import shutil

for mushroomName in mushroomNames:
    dirListing = os.listdir('../../MushroomImage/images_nobg/' + mushroomName)
    if(len(dirListing) < 15):
        try:
            shutil.rmtree('../../MushroomImage/images_nobg/' + mushroomName)
        except OSError as e:
            print("Error: %s : %s" % (dir_path, e.strerror))
    print(len(dirListing))




mushroomNames = []
for mushroomName in os.listdir('../../MushroomImage/' + 'images_qual/'):
    mushroomNames.append(mushroomName)

for mushroomName in mushroomNames[:]:
    for imageName in os.listdir('../../MushroomImage/' + 'images_qual/' + mushroomName):
        image = cv2.imread('../../MushroomImage/' + 'images_qual/' + mushroomName + '/' + imageName, cv2.IMREAD_COLOR)
        try:
            image = cv2.resize(image, (150,150))
        except:
            break
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        rectangle = (10,10,130,130)
        mask = np.zeros(image.shape[:2], np.uint8)
        bgdModel = np.zeros((1,65), np.float64)
        fgdModel = np.zeros((1,65), np.float64)
        
        cv2.grabCut(image,
                   mask,
                   rectangle,
                   bgdModel,
                   fgdModel,
                   5,
                   cv2.GC_INIT_WITH_RECT)
        
        mask_2 = np.where((mask==2) | (mask==0), 0, 1).astype('uint8')
    
        image_nobg = image * mask_2[:,:,np.newaxis]
        sparsity = 1.0 - np.count_nonzero(image_nobg) / image_nobg.size
        plt.imshow(image_nobg)
        if(sparsity < 0.95):
            cv2.imwrite('../../MushroomImage/' + file_under_use  + mushroomName + '/' + imageName, image_nobg)
            print(mushroomName)
        #cv2.imwrite('../../MushroomImage/' + file_under_use  + mushroomName + '/' + imageName, image)
print(image.flatten().shape)